<a href="https://colab.research.google.com/github/abishekraja018/SDC-GENAI/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ LSTM Character Predictor in Google Colab

# Step 1: Imports and Device Setup
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 2: Sample Sequence and Preprocessing
sequence = "hello"
chars = list(set(sequence))  # Unique characters
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for ch, i in char_to_idx.items()}

input_seq = [char_to_idx[ch] for ch in sequence[:-1]]  # 'h', 'e', 'l', 'l'
target_seq = [char_to_idx[ch] for ch in sequence[1:]]  # 'e', 'l', 'l', 'o'

# One-hot encoding input
input_tensor = torch.eye(len(chars))[input_seq].unsqueeze(1).to(device)  # Shape: (seq_len, batch, input_size)
target_tensor = torch.tensor(target_seq).to(device)

# Step 3: Define the LSTM Model
class CharLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, (hidden, cell) = self.lstm(x)
        out = self.fc(out.view(-1, out.shape[2]))  # Flatten and pass through FC
        return out

# Step 4: Initialize and Train the Model
input_size = len(chars)
hidden_size = 16
output_size = len(chars)
num_epochs = 100

model = CharLSTM(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

print("\nTraining...\n")
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(input_tensor)
    loss = criterion(output, target_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        pred = output.argmax(dim=1)
        pred_str = ''.join([idx_to_char[i.item()] for i in pred])
        print(f"Epoch {epoch+1:03}, Loss: {loss.item():.4f}, Prediction: {pred_str}")

print("\n✅ Training Complete!")


Using device: cpu

Training...

Epoch 010, Loss: 1.1302, Prediction: llll
Epoch 020, Loss: 0.7786, Prediction: lllo
Epoch 030, Loss: 0.4062, Prediction: lllo
Epoch 040, Loss: 0.1789, Prediction: ello
Epoch 050, Loss: 0.0777, Prediction: ello
Epoch 060, Loss: 0.0363, Prediction: ello
Epoch 070, Loss: 0.0207, Prediction: ello
Epoch 080, Loss: 0.0138, Prediction: ello
Epoch 090, Loss: 0.0101, Prediction: ello
Epoch 100, Loss: 0.0080, Prediction: ello

✅ Training Complete!
